In [1]:
import pandas as pd
import ast


filename = './healsl_rd2_rapid_gpt4_v2b_sorted_ICD.csv'
df = pd.read_csv(filename)

df.output_probs = df.output_probs.apply(ast.literal_eval)

In [2]:
if "gpt3" in filename:
    model_name = "GPT3"
elif "gpt4"in filename:
    model_name = "GPT4"

if "first_ICD" in filename:
    sort_scheme = "First ICD"
    test_sort = False
elif "sorted_ICD"  in filename:
    sort_scheme = "Highest Prob"
    test_sort = True
    
if "rd1" in filename:
    round_num = "Round 1"
elif "rd2" in filename:
    round_num = "Round 2"
    
if "sample" in filename:
    sample = "Sample"
else:
    sample = "Full"
    

In [3]:
def prob_ICD(probs, sort=False):
    if probs == []:
        return []
    temp = pd.DataFrame(probs)    
    
    if sort:
        temp = temp.sort_values(by='icd_linprob_mean', ascending=False)
    
    # display(temp)
    return temp.icd.to_list()

In [4]:
df['cause_list'] = df[['cause1_icd10', 'cause2_icd10', 'cause3_icd10', 'cause4_icd10', 'cause5_icd10']].apply(lambda x: x.dropna().tolist(), axis=1)
df['sanity'] = df.output_probs.apply(prob_ICD, sort=test_sort)


In [5]:
print(f"Model: {model_name}, Sort: {sort_scheme}, {round_num}")
print(f"Shape: {df.shape}")

error_flag = False
for idx, a in df[['cause_list', 'sanity']].iterrows():
    if a.cause_list[:5] != a.sanity[:5]:
        print(idx)
        print(a.cause_list)
        print(a.sanity)
        error_flag = True
        
if error_flag:
    print("Mismatch found!")
else:
    print("No mismatch found!")
    

Model: GPT4, Sort: Highest Prob, Round 2
Shape: (3317, 25)
No mismatch found!


In [6]:
# df.loc[3280].output_probs